<a href="https://colab.research.google.com/github/Remon128/Arabic_Dialect_Sentiment_Analysis/blob/main/model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy  as np
from gensim.models import word2vec
import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install important libs
!pip3 install --upgrade gensim

In [ ]:
# Loading models
word2vec_model = word2vec.Word2Vec()
word2vec_model = word2vec_model.wv.load("/content/drive/MyDrive/Colab Notebooks/Arabic-Sentence-Ident/word2vec_twitter_Model")

In [ ]:
word2vec_model.most_similar('قول')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Arabic-Sentence-Ident/dialect_dataset_v_prep.csv", index_col=[0])

In [ ]:
# Loading arabert App

# Cloning and installing arabert dependencies
#change dir to project dir
%cd "/content/drive/MyDrive/Colab Notebooks/Arabic-Sentence-Ident"
#clone arabert library
!git clone "https://github.com/aub-mind/arabert.git"
#install farasapy dependencies
!pip install farasapy
#install pyarabic dependencies
!pip install pyarabic

In [ ]:
from arabert.preprocess import ArabertPreprocessor

In [ ]:
model_name = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name, apply_farasa_segmentation= True)

In [ ]:
# Testing arabert model
text = "@3lgwEDrtQnddUUp @Mohassad71 والله الخبر عند ص @ha___m___ed والله العظيم ضحكتني  ي"
arabert_prep.preprocess(text)

'[مستخدم] [مستخدم] و+ الله ال+ خبر عند ص [مستخدم] و+ الله ال+ عظيم ضحكتني ي'

In [ ]:
#Insall Flask Important Libraries
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   
              

In [ ]:
# Flask ngrok authentication
!ngrok authtoken 269LPeiVyUqSTONcwiIctCXoyjq_54JtqeR6C7MUoMyChx3fc

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Loading Saved Tokens and Models
token_file =  open('/content/drive/MyDrive/Colab Notebooks/Arabic-Sentence-Ident/Tokenizer.pickle', 'rb')
tokens = pickle.load(token_file)
bi_lstm_Model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Arabic-Sentence-Ident/Sentiment-BiLSTM')

In [ ]:
# Load and Predict Output
def load_and_Predict_Input(Input,tokens, model):
  test = pad_sequences(tokens.texts_to_sequences(input) , maxlen=100)
  y_pred = bi_lstm_Model.predict(test)
  i, j = np.unravel_index(np.argmax(y_pred), y_pred.shape)
  labels_name = data['Dialect'].unique()
  pred = labels_name[j]
  return pred

In [ ]:
input = " ازيك عامل ايه طمني عليك؟"
load_and_Predict_Input(input, tokens, bi_lstm_Model)

'PL'

In [ ]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok

In [ ]:
# Creating Flask App

app = Flask(__name__, template_folder="/content/drive/MyDrive/Colab Notebooks/Arabic-Sentence-Ident/templates")
#dT_model = pickle.load(open('Descion_Tree_Model.pickle', 'rb'))
run_with_ngrok(app)
#render_template('index.html')
@app.route('/')
def home():
    return render_template('predict.html')

@app.route('/Dialect_Predict', methods=['POST'])
def Dialect_Predict():
    input = [x for x in request.form.values()]
    final_input = [np.array(input)]
    predict = load_and_Predict_Input(final_input[0][0], tokens, bi_lstm_Model)
    return render_template('predict.html', output='Predicted Dialect: {}'.format(predict))
    

if __name__ == "__main__":
  app.debug
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1db8-34-125-186-29.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/Mar/2022 17:44:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 17:44:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Mar/2022 17:44:40] "POST /Dialect_Predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 17:49:23] "GET / HTTP/1.1" 200 -
